In [2]:
import pandas as pd
import ast
import asyncio
import os
from typing import List, Dict
from dotenv import load_dotenv
from openai import AsyncOpenAI
from tqdm.asyncio import tqdm_asyncio
import json
import random

# COT PARSING

In [21]:
with open("webglm_full_cot.jsonl", "r") as input, \
     open("webglm_full_cot_parsed.jsonl", "w") as output:
    
    for idx, line in enumerate(input):
    
        data = json.loads(line)
        cot_answer = data['answer']
        
        parts = cot_answer.split('ОТВЕТ:', 1)

        if len(parts) != 2:
            continue
        
        cot = parts[0].strip() if len(parts) > 0 else ''
        answer = parts[1].strip() if len(parts) > 1 else ''
        
        data['cot'] = cot
        data['answer'] = answer
        
        json_line = json.dumps(data)
        output.write(json_line + '\n')

In [11]:
# Убрали 100 строк с мусорной разметкой
with open("webglm_full_cot_parsed.jsonl", "r") as input:
    print(len(input.readlines()))

43487


In [26]:
with open("webglm_full_cot_parsed.jsonl", "r") as file:
    row = json.loads(file.readlines()[5071])

print(row['question'])
print('\n\n')
print(row['cot'])
print('\n\n')
print(row['answer'])
print('\n\n')
print(row['references'])

Что происходит с батареями, когда они перестают работать?



Когда пользователь задает вопрос о том, что происходит с батареями, когда они перестают работать, он, вероятно, ищет информацию о процессе, который приводит к потере заряда и возможности повторного использования или переработки батарей. В источнике [1] упоминается, что все батареи в конечном итоге теряют заряд, но не предоставляется конкретного объяснения этого процесса. Однако в источнике [5] более подробно описывается причина, по которой литий-ионные батареи теряют заряд с течением времени, что связано с нежелательной химической реакцией, начинающейся с электродов, содержащих никель.

Эта химическая реакция, по-видимому, является ключевым фактором в потере заряда батарей. Кроме того, источник [2] затрагивает тему того, что происходит с аккумуляторами электромобилей, когда они достигают конца своего срока службы, что подразумевает, что существует процесс, связанный с их конечным состоянием. Более конкретно, источник [3] упом

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B")

question_tokens = 0
cot_tokens = 0
answer_tokens = 0
context_tokens = 0

with open("webglm_full_cot_parsed.jsonl", "r") as file:
    samples = len(file.readlines())

with open("webglm_full_cot_parsed.jsonl", "r") as file:
    for idx, line in enumerate(file):
        data = json.loads(line)

        question_tokens += len(tokenizer.encode(data['question'], add_special_tokens=False))
        cot_tokens += len(tokenizer.encode(data['cot'], add_special_tokens=False))
        answer_tokens += len(tokenizer.encode(data['answer'], add_special_tokens=False))

        context = ''
        for i, c in enumerate(data['references']):
            context += f'Источник [{i+1}]:'+"\n"+c+"\n\n"
        context_tokens += len(tokenizer.encode(context, add_special_tokens=False))


print(f"question {question_tokens / samples}")
print(f"cot {cot_tokens / samples}")
print(f"answer {answer_tokens / samples}")
print(f"context {context_tokens / samples}")

question 25.323292018304322
cot 492.39370386552304
answer 237.87941223814013
context 517.8336514360614


# DEFAULT WEBGLM

In [3]:
SYSTEM_PROMPT = (
    "Ты — экспертная система Compressa RAG, "
    "предоставляющая точные и релевантные ответы на вопросы, "
    "используя только предоставленную контекстную информацию. "
    "Отвечай на русском языке."
)
REJECT_ANSW = "К сожалению, ответа на вопрос нет в упомянутых источниках"

def get_summary_prompt(context_list, question):
    context = ''
    for i, c in enumerate(context_list):
        context += f'Источник [{i+1}]:'+"\n"+c+"\n\n"

    prompt = (
        f"# Контекстная информация:\n\n{context}\n\n"
        "---\n"
        "# Инструкции:\n\n"
        "1. Дай полный и краткий ответ на вопрос, используя только информацию из контекста.\n"
        "2. Укажи номер источника в квадратных скобках после использовании фактов из него, например: [1].\n"
        "3. Ответ должен содержать хотя бы одну ссылку на источник.\n"
        "4. Ссылайся на источник только если информация взята из него.\n"
        f"5. Если ответа на вопрос нет в источниках, ответь: \"{REJECT_ANSW}\".\n"
        "6. Не используй знания вне предоставленного контекста.\n\n"
        f"# Вопрос:\n\n{question}\n\n"
    )
    return prompt

def get_raft_summary_prompt(context_list, question):
    context = ''
    for i, c in enumerate(context_list):
        context += f'Источник [{i+1}]:'+"\n"+c+"\n\n"

    prompt = (
        f"# Контекстная информация:\n\n{context}\n\n"
        "---\n"
        "# Инструкции:\n\n"
        "1. Сперва порассуждай про то, какую информацию хочет узнать пользователь" 
        "и как информация из контекста с этим соотносится.\n"
        "2. В последнем абзаце дай полный и краткий ответ на вопрос, используя только информацию из контекста.\n"
        "3. Укажи номер источника в квадратных скобках после использовании фактов из него, например: [1].\n"
        "4. Ответ должен содержать хотя бы одну ссылку на источник.\n"
        "5. Ссылайся на источник только если информация взята из него.\n"
        f"6. Если ответа на вопрос нет в источниках, ответь: \"{REJECT_ANSW}\".\n"
        "7. Не используй знания вне предоставленного контекста.\n\n"
        f"# Вопрос:\n\n{question}\n\n"
    )
    return prompt

def update_refs_number(answer, old_numbers, new_numbers):
    for old, new in zip(old_numbers, new_numbers):
        answer = answer.replace(f"[{old}]", f"[{new}]")
    return answer

In [20]:
df = pd.read_excel('webglm_full.xlsx')

def create_llama_dataset(df: pd.DataFrame) -> List[Dict]:
    dataset = []
    for _, row in df.iterrows():
        question = row['question']
        answer = str(row['answer'])
        examples = ast.literal_eval(row["references"]) if isinstance(row['references'], str) else []
        
        dataset_item = {
            "question": question,
            "answer": answer,
            "references": examples
        }
        dataset.append(dataset_item)
    return dataset


dataset = create_llama_dataset(df)
len(dataset)

43579

In [ ]:
with open("webglm.jsonl", "w") as file:
    for row in dataset:
        data = ({
            "messages": [
                {"content": SYSTEM_PROMPT, "role": "system"},
                {"content": get_summary_prompt(row["references"], row["question"]), "role": "user"},
                {"content": row["answer"], "role": "assistant"}
            ],
            "opus_score": 10,
        })
        
        json.dump(data, file)
        file.write('\n')

# RAFT

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B")

In [5]:
from queue import deque

max_refs = 4
storage = deque(maxlen=max_refs)

with open("webglm_full_cot_parsed.jsonl", "r") as f:
    for line in f:
        storage.append(json.loads(line))

In [6]:
with open("webglm_full_cot_parsed.jsonl", "r") as input, \
     open("webglm_raft.jsonl", "w") as output:  
    for i, row in enumerate(input):
        line = json.loads(row)
        true_refs = line["references"]
        all_refs = []
        all_refs += true_refs

        for data in storage:
            all_refs += data["references"]

        random.shuffle(all_refs)
        new_numbers = [all_refs.index(ref) + 1 for ref in true_refs]
        answer = update_refs_number(line["answer"], list(range(1, len(new_numbers)+1)), new_numbers)
        cot = update_refs_number(line["cot"], list(range(1, len(new_numbers)+1)), new_numbers)

        storage.append(line)
        
        data = ({
            "messages": [
                {"content": SYSTEM_PROMPT, "role": "system"},
                {"content": get_raft_summary_prompt(all_refs, line["question"]), "role": "user"},
                {"content": cot + "\n\n" + answer, "role": "assistant"}
            ],
            "opus_score": 10,
        })

        if len(tokenizer.encode(tokenizer.apply_chat_template(data["messages"], tokenize=False), add_special_tokens=False)) > 5000:
            continue

        json.dump(data, output)
        output.write('\n')

In [7]:
max_refs = 4
storage = deque(maxlen=max_refs)

with open("webglm_full_cot_parsed.jsonl", "r") as f:
    for line in f:
        storage.append(json.loads(line))

In [8]:
with open("webglm_full_cot_parsed.jsonl", "r") as input, \
     open("webglm_raft.jsonl", "a") as output:
    for idx, line in enumerate(input):
        if idx == 5000:
            break

        question = json.loads(line)["question"]
        
        all_refs = []

        for data in storage:
            all_refs += data["references"]

        random.shuffle(all_refs)

        data = ({
            "messages": [
                {"content": SYSTEM_PROMPT, "role": "system"},
                {"content": get_raft_summary_prompt(all_refs, question), "role": "user"},
                {"content": REJECT_ANSW, "role": "assistant"}
            ],
            "opus_score": 10,
        })

        storage.append(json.loads(line))

        if len(tokenizer.encode(tokenizer.apply_chat_template(data["messages"], tokenize=False), add_special_tokens=False)) > 5000:
            continue


        json.dump(data, output)
        output.write('\n')

In [1]:
with open("webglm_raft.jsonl", "r") as input:
    print(len(input.readlines()))

48468


In [10]:
with open("webglm_raft.jsonl", 'r') as input, \
     open("webglm_raft_shuffled.jsonl", 'w') as output:
    lines = input.readlines()
    
    random.shuffle(lines)

    output.writelines(lines)

In [11]:
with open("webglm_raft_shuffled.jsonl", "r") as input:
    print(len(input.readlines()))

48468
